## notebook to combine all the data, from CAMS, ERA5, feature engineering and others

In [1]:
## conda environment Weather_Prediction

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import geopandas as gpd
import xarray as xr
pd.set_option('display.max_columns', None)

## Before we merge, lets find out counties whose shapefile does not exist, and those counties whose shapefiles does exist but the life expectancy data doesnot exist


In [2]:
## Lets load the county shapefile

SHAPE_PATH = os.path.join('../Shapefiles','county_shapefiles','2010_county_shapefile','gz_2010_us_050_00_500k.shp')
county_gdf = gpd.read_file(SHAPE_PATH)

## lets create the fips column in the dataframe which is the sum of state and county both as dtype string.
## Note: not a numerical sum
## This step is necessary because there can be several counties with idential names. 

county_gdf['fips']=county_gdf['STATE'] + county_gdf['COUNTY']

## convert the data type
county_gdf['fips']=county_gdf['fips'].astype(str).astype(int)

county_gdf = county_gdf.drop([ 'GEO_ID','CENSUSAREA','COUNTY','LSAD'], axis=1)
county_gdf

,STATE,NAME,geometry,fips
0,01,Cleburne,"POLYGON ((-85.38872 33.91304, -85.38088 33.873...",1029
1,01,Coffee,"POLYGON ((-86.03044 31.61894, -86.00408 31.619...",1031
2,01,Coosa,"POLYGON ((-86.00928 33.10164, -86.00917 33.090...",1037
3,01,Covington,"POLYGON ((-86.34851 30.99434, -86.35023 30.994...",1039
4,01,Crenshaw,"POLYGON ((-86.14699 31.68045, -86.14711 31.663...",1041
...,...,...,...,...
3216,72,San Sebastián,"POLYGON ((-66.90748 18.25314, -66.90739 18.253...",72131
3217,72,Santa Isabel,"POLYGON ((-66.37968 17.94398, -66.38029 17.943...",72133
3218,72,Toa Baja,"MULTIPOLYGON (((-66.13957 18.46232, -66.13956 ...",72137
3219,72,Trujillo Alto,"POLYGON ((-66.02917 18.37590, -66.02828 18.376...",72139


In [3]:
## gather only state fips of USA not its territories such as guam, puerto rico

county_gdf['STATE']=county_gdf['STATE'].astype(str).astype(int) ## convert state dtype to int
county_gdf=county_gdf.loc[(county_gdf['STATE'] < 60)]
county_gdf

,STATE,NAME,geometry,fips
0,1,Cleburne,"POLYGON ((-85.38872 33.91304, -85.38088 33.873...",1029
1,1,Coffee,"POLYGON ((-86.03044 31.61894, -86.00408 31.619...",1031
2,1,Coosa,"POLYGON ((-86.00928 33.10164, -86.00917 33.090...",1037
3,1,Covington,"POLYGON ((-86.34851 30.99434, -86.35023 30.994...",1039
4,1,Crenshaw,"POLYGON ((-86.14699 31.68045, -86.14711 31.663...",1041
...,...,...,...,...
3138,56,Niobrara,"POLYGON ((-104.05298 42.85955, -104.05286 42.7...",56027
3139,56,Platte,"POLYGON ((-104.77417 42.60996, -104.76422 42.6...",56031
3140,56,Sweetwater,"POLYGON ((-109.05008 41.00066, -109.17368 41.0...",56037
3141,56,Washakie,"POLYGON ((-107.12892 43.99455, -107.12797 43.9...",56043


In [4]:
column_values = county_gdf['STATE'].unique()
column_values

array([ 1,  2,  4,  5,  6,  8,  9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20,
       21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
       38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56])

In [5]:
## The life expectency data

PATH = os.path.join('../life_expectancy_data','LE_2010.csv')
LE_DATA = pd.read_csv(PATH)
LE_DATA

,location_name,fips,year,MeanLifeExpectency
0,Autauga County (Alabama),1001,2010,75.728489
1,Baldwin County (Alabama),1003,2010,77.826608
2,Barbour County (Alabama),1005,2010,75.841973
3,Bibb County (Alabama),1007,2010,73.705432
4,Blount County (Alabama),1009,2010,75.753407
...,...,...,...,...
3122,Sweetwater County (Wyoming),56037,2010,77.582422
3123,Teton County (Wyoming),56039,2010,83.771012
3124,Uinta County (Wyoming),56041,2010,77.836526
3125,Washakie County (Wyoming),56043,2010,78.674180


In [6]:
## the counties that are in life expectency data but the shape file does not exist

county_noshape = LE_DATA.loc[~LE_DATA['fips'].isin(county_gdf['fips'])].copy()
county_noshape

,location_name,fips,year,MeanLifeExpectency
80,Chugach Census Area (Alaska),2063,2010,77.733009
81,Copper River Census Area (Alaska),2066,2010,77.733009
83,Kusilvak Census Area (Alaska),2158,2010,69.723907
85,Kobuk Census Area (Alaska),2140,2010,73.339132
87,Aleutian Islands Census Area (Alaska),2010,2010,82.631153
96,Skagway-Yakutat-Angoon Census Area (Alaska),2231,2010,80.202830
97,Skagway-Hoonah-Angoon Census Area (Alaska),2232,2010,80.202830
102,Prince of Wales-Outer Ketchikan Census Area (A...,2201,2010,78.068610
104,Wrangell-Petersburg Census Area (Alaska),2280,2010,78.068610
338,Dade County (Florida),12025,2010,80.576828


## Note that, among the 3127 total counties, 38 are that of Alaska, 5 are that of Hawaii and as shown above there are 6 counties where the life expectancy data does not exist. In total, 3127-5-6= 3078 counties 

In [7]:
## the counties that are in shapefile data but life expectancy data does not exist

le_data_nodata = county_gdf.loc[~county_gdf['fips'].isin(LE_DATA['fips'])].copy()
le_data_nodata

,STATE,NAME,geometry,fips
244,8,Hinsdale,"POLYGON ((-107.21157 37.42296, -107.21787 37.4...",8053
250,8,Mineral,"POLYGON ((-106.71077 37.40423, -106.71077 37.3...",8079
290,8,San Juan,"POLYGON ((-107.92580 37.63979, -107.92613 37.6...",8111
585,16,Clark,"POLYGON ((-112.81416 44.37720, -112.81324 44.3...",16033
1625,30,Petroleum,"POLYGON ((-108.50506 46.74914, -108.63095 46.7...",30069
1640,30,Treasure,"POLYGON ((-107.05209 45.87700, -107.05203 45.8...",30103
1645,30,Golden Valley,"POLYGON ((-109.31590 46.75150, -109.26339 46.7...",30037
1666,31,Grant,"POLYGON ((-101.62042 41.74240, -101.63796 41.7...",31075
1667,31,Hooker,"POLYGON ((-101.41220 42.09211, -101.37545 42.0...",31091
1669,31,Logan,"POLYGON ((-100.71171 41.73976, -100.50060 41.7...",31113


## Lets start combining data

In [8]:
## load the data with county and corresponding variables

county_var=pd.read_pickle('CAMS_79_variables.pkl')
county_var

,NAME,fips,u10,v10,d2m,t2m,bcaod550,duaod550,lsm,msl,omaod550,pm1,pm2p5,pm10,ssaod550,suaod550,z,sp,aod469,aod550,aod670,aod865,aod1240,tcco,tc_c2h6,tchcho,tc_h2o2,tc_oh,tc_c5h8,tc_ch4,tc_hno3,tcno2,tc_no,gtco3,tc_pan,tc_c3h8,tcso2,tcwv,co,aermr04,aermr05,aermr06,c2h6,hcho,h2o2,aermr09,aermr07,aermr10,aermr08,oh,c5h8,hno3,no2,no,go3,pan,c3h8,aermr01,aermr02,aermr03,q,aermr11,so2,t,lai_hv,lai_lv,asn,sd,tcw,aermssdus,aermssdum,aermssdul,aermssbchphil,aermssomhphil,aermssbchphob,aermssomhphob,aermsssss,aermssssm,aermssssl,aermsssu,aermssso2
0,Cleburne,1029,0.434079,-0.122697,282.638367,289.807770,0.006281,0.007998,0.985105,101706.890625,0.077957,1.335776e-08,1.673881e-08,2.318515e-08,0.006267,0.078404,2581.377930,98584.976562,0.216394,0.176908,0.135071,0.092637,0.051929,0.000884,0.000003,0.000004,0.000008,4.903662e-09,2.851306e-06,0.009726,0.000005,0.000004,5.587670e-07,0.006659,0.000010,7.175567e-07,4.614676e-06,23.744879,1.941375e-07,5.623386e-10,9.944396e-10,3.588945e-10,6.390680e-10,3.820059e-09,8.087090e-10,2.126177e-10,8.745088e-09,2.581321e-10,5.042643e-09,2.414235e-14,2.003694e-08,9.753656e-10,1.124590e-08,2.180646e-09,5.659882e-08,2.960079e-09,3.714545e-10,1.811105e-11,1.448672e-09,1.733458e-10,0.008866,3.393259e-09,1.341741e-08,289.580719,3.637702,2.688559,0.877579,0.007665,23.808104,1.562752e-06,0.000003,0.000002,1.405128e-07,0.000003,5.258444e-07,0.000013,1.997616e-07,0.000013,7.354205e-07,0.000009,3.305535e-06
1,Coffee,1031,0.277523,-0.034499,284.563934,291.594208,0.006543,0.012055,0.992762,101687.679688,0.073746,1.180315e-08,1.550131e-08,2.120308e-08,0.008770,0.067104,803.111572,100711.117188,0.204075,0.168218,0.130380,0.092329,0.054776,0.000880,0.000003,0.000004,0.000009,4.889060e-09,3.093836e-06,0.009946,0.000005,0.000003,4.301936e-07,0.006470,0.000009,6.683903e-07,2.240001e-06,26.696531,1.820293e-07,8.003456e-10,1.406093e-09,5.522706e-10,5.869064e-10,4.012024e-09,8.954773e-10,2.282372e-10,7.808930e-09,3.169671e-10,4.456306e-09,2.260730e-14,1.899532e-08,8.763487e-10,6.403306e-09,3.752219e-10,5.846175e-08,2.845826e-09,2.449791e-10,2.979960e-11,2.399912e-09,4.469469e-10,0.010002,1.925459e-09,1.983883e-09,291.643219,3.733347,2.400462,0.879168,0.005595,26.736628,2.329077e-06,0.000005,0.000004,1.444494e-07,0.000002,5.499585e-07,0.000013,2.590336e-07,0.000018,1.376015e-06,0.000008,9.460180e-07
2,Coosa,1037,0.262713,-0.041072,283.402100,290.892761,0.006274,0.010251,0.984186,101696.562500,0.076223,1.158120e-08,1.503791e-08,2.065578e-08,0.007309,0.073413,1495.930664,99881.492188,0.211330,0.173469,0.133421,0.092969,0.053567,0.000888,0.000003,0.000004,0.000009,4.881469e-09,3.799016e-06,0.009860,0.000005,0.000003,4.759287e-07,0.006597,0.000009,6.958822e-07,3.450811e-06,25.366951,1.793285e-07,7.114352e-10,1.270793e-09,4.720206e-10,5.698053e-10,4.127878e-09,8.693078e-10,1.921167e-10,7.605931e-09,2.044141e-10,4.114898e-09,2.056732e-14,2.135452e-08,8.684681e-10,7.947802e-09,7.347526e-10,5.876263e-08,3.034337e-09,2.672530e-10,2.268106e-11,1.816531e-09,2.506161e-10,0.009254,2.694272e-09,7.586452e-09,290.732147,3.872691,2.624079,0.878558,0.006002,25.419048,1.995773e-06,0.000004,0.000003,1.306877e-07,0.000003,5.350508e-07,0.000013,2.238712e-07,0.000015,9.479429e-07,0.000008,2.127009e-06
3,Covington,1039,0.219254,0.026002,284.972931,291.764221,0.006611,0.012473,0.938187,101689.382812,0.074370,1.182135e-08,1.562129e-08,2.134704e-08,0.009069,0.068461,763.528442,100760.820312,0.207274,0.170984,0.132668,0.094133,0.055998,0.000879,0.000003,0.000004,0.000009,4.894909e-09,3.072844e-06,0.009951,0.000005,0.000003,4.309889e-07,0.006472,0.000009,6.685116e-07,2.259529e-06,26.823252,1.783561e-07,8.262079e-10,1.468629e-09,5.814572e-10,5.805680e-10,3.841004e-09,9.165831e-10,2.354176e-10,7.717706e-09,3.395666e-10,4.358559e-09,2.310399e-14,1.779057e-08,9.000244e-10,6.136765e-09,3.290059e-10,6.028979e-08,2.792559e-09,2.386336e-10,3.287333e-11,2.653059e-09,5.576852e-10,0.010171,2.021648e-09,2.193981e-09,291.732147,4.064345,2.598899,0.879063,0.005539,26.

In [9]:
## The life expectency data

PATH = os.path.join('../life_expectancy_data','LE_2010.csv')
LE_DATA = pd.read_csv(PATH)
LE_DATA

,location_name,fips,year,MeanLifeExpectency
0,Autauga County (Alabama),1001,2010,75.728489
1,Baldwin County (Alabama),1003,2010,77.826608
2,Barbour County (Alabama),1005,2010,75.841973
3,Bibb County (Alabama),1007,2010,73.705432
4,Blount County (Alabama),1009,2010,75.753407
...,...,...,...,...
3122,Sweetwater County (Wyoming),56037,2010,77.582422
3123,Teton County (Wyoming),56039,2010,83.771012
3124,Uinta County (Wyoming),56041,2010,77.836526
3125,Washakie County (Wyoming),56043,2010,78.674180


In [10]:
## merge life expectency data with the variables by fips

final_df=LE_DATA.merge(county_var,how='left', on='fips')
final_df=final_df.dropna()
final_df

,location_name,fips,year,MeanLifeExpectency,NAME,u10,v10,d2m,t2m,bcaod550,duaod550,lsm,msl,omaod550,pm1,pm2p5,pm10,ssaod550,suaod550,z,sp,aod469,aod550,aod670,aod865,aod1240,tcco,tc_c2h6,tchcho,tc_h2o2,tc_oh,tc_c5h8,tc_ch4,tc_hno3,tcno2,tc_no,gtco3,tc_pan,tc_c3h8,tcso2,tcwv,co,aermr04,aermr05,aermr06,c2h6,hcho,h2o2,aermr09,aermr07,aermr10,aermr08,oh,c5h8,hno3,no2,no,go3,pan,c3h8,aermr01,aermr02,aermr03,q,aermr11,so2,t,lai_hv,lai_lv,asn,sd,tcw,aermssdus,aermssdum,aermssdul,aermssbchphil,aermssomhphil,aermssbchphob,aermssomhphob,aermsssss,aermssssm,aermssssl,aermsssu,aermssso2
0,Autauga County (Alabama),1001,2010,75.728489,Autauga,0.244891,-0.022881,283.480743,291.070984,0.006298,0.010663,0.986290,101694.718750,0.076191,1.131367e-08,1.479475e-08,2.029277e-08,0.007522,0.072511,1337.441284,100070.937500,0.210819,0.173185,0.133389,0.093220,0.053980,0.000888,0.000003,4.347277e-06,0.000009,4.880845e-09,3.887436e-06,0.009879,0.000005,0.000003,4.690559e-07,0.006586,0.000009,6.928641e-07,3.153174e-06,25.591148,1.778901e-07,7.377920e-10,1.319926e-09,4.948856e-10,5.604479e-10,4.120476e-09,8.757213e-10,1.897001e-10,7.463049e-09,1.960385e-10,3.993148e-09,2.022502e-14,2.141237e-08,8.628354e-10,7.536546e-09,6.557893e-10,5.894945e-08,3.017641e-09,2.562929e-10,2.378038e-11,1.905661e-09,2.728035e-10,0.009295,2.539864e-09,6.091789e-09,290.919830,3.897127,2.624408,0.878641,0.005784,25.642176,2.074938e-06,4.218031e-06,3.245817e-06,1.298927e-07,0.000003,5.384176e-07,0.000013,2.288023e-07,0.000016,9.983653e-07,0.000008,1.850165e-06
1,Baldwin County (Alabama),1003,2010,77.826608,Baldwin,0.065638,0.113299,286.096985,292.149780,0.006543,0.013571,0.793853,101686.296875,0.074374,1.082756e-08,1.474964e-08,2.021521e-08,0.010043,0.070549,426.965088,101166.343750,0.211824,0.175080,0.136272,0.097289,0.058477,0.000877,0.000003,3.707628e-06,0.000009,4.924390e-09,2.554849e-06,0.009991,0.000005,0.000003,4.425227e-07,0.006451,0.000008,6.634551e-07,2.463177e-06,27.551956,1.661377e-07,8.725340e-10,1.541212e-09,6.062684e-10,5.508705e-10,3.224980e-09,9.064102e-10,2.039385e-10,6.913703e-09,2.441078e-10,3.649391e-09,2.652614e-14,1.445367e-08,9.667648e-10,6.547783e-09,5.070752e-10,6.086813e-08,2.466012e-09,2.234096e-10,4.787582e-11,3.910108e-09,1.548187e-09,0.010791,2.249294e-09,3.825508e-09,292.066528,3.845286,2.597180,0.879368,0.005067,27.597118,2.609881e-06,5.381241e-06,4.267411e-06,1.304158e-07,0.000002,5.643739e-07,0.000013,2.814684e-07,0.000020,1.889070e-06,0.000008,1.206832e-06
2,Barbour County (Alabama),1005,2010,75.841973,Barbour,0.336253,-0.075642,284.153351,291.171997,0.006456,0.010674,0.987881,101689.679688,0.074089,1.173599e-08,1.520591e-08,2.082824e-08,0.007945,0.067335,1178.228027,100259.171875,0.202453,0.166499,0.128529,0.090249,0.052785,0.000884,0.000003,4.157243e-06,0.000009,4.883077e-09,3.436399e-06,0.009900,0.000005,0.000003,4.421493e-07,0.006519,0.000009,6.808563e-07,2.526453e-06,26.079842,1.840146e-07,7.171533e-10,1.263283e-09,4.802824e-10,5.927475e-10,4.009649e-09,8.788015e-10,2.286225e-10,7.826896e-09,3.231082e-10,4.440267e-09,2.134459e-14,2.074101e-08,8.272189e-10,6.289255e-09,3.875605e-10,5.690426e-08,2.879866e-09,2.500995e-10,2.469199e-11,1.982629e-09,3.230463e-10,0.009707,2.009621e-09,2.205164e-09,291.179504,4.070218,2.398671,0.878966,0.006158,26.123323,2.068363e-06,4.211733e-06,3.332815e-06,1.467424e-07,0.000002,5.382911e-07,0.000013,2.391392e-07,0.000017,1.143327e-06,0.000007,1.193010e-06
3,Bibb County (Alabama),1007,2010,73.705432,Bibb,0.221962,-0.004751,283.564850,290.792206,0.006248,0.010553,0.986778,101695.648438,0.077052,1.129947e-08,1.473398e-08,2.021792e-08,0.007260,0.072814,1267.733643,100154.929688,0.211800,0.173926,0.133868,0.093414,0.053924,0.000892,0.000003,4.335408e-06,0.000009,4.883221e-09,3.874136e-06,0.009887,0.000005,0.000003,4.890971e-07,0.006616,0.000009,6.987318e-07,3.148527e-06,25.509922,1.782282e-07,7.250749e-10,1.290446e-09,4.728918e-10,5.571377e-10,4.112687e-09,8.437939e-10,1.833033e-10,7.471534e-09,1.787300e-10,3.9769

In [11]:
final_df = final_df.drop(['location_name', 'NAME'], axis=1)
final_df

,fips,year,MeanLifeExpectency,u10,v10,d2m,t2m,bcaod550,duaod550,lsm,msl,omaod550,pm1,pm2p5,pm10,ssaod550,suaod550,z,sp,aod469,aod550,aod670,aod865,aod1240,tcco,tc_c2h6,tchcho,tc_h2o2,tc_oh,tc_c5h8,tc_ch4,tc_hno3,tcno2,tc_no,gtco3,tc_pan,tc_c3h8,tcso2,tcwv,co,aermr04,aermr05,aermr06,c2h6,hcho,h2o2,aermr09,aermr07,aermr10,aermr08,oh,c5h8,hno3,no2,no,go3,pan,c3h8,aermr01,aermr02,aermr03,q,aermr11,so2,t,lai_hv,lai_lv,asn,sd,tcw,aermssdus,aermssdum,aermssdul,aermssbchphil,aermssomhphil,aermssbchphob,aermssomhphob,aermsssss,aermssssm,aermssssl,aermsssu,aermssso2
0,1001,2010,75.728489,0.244891,-0.022881,283.480743,291.070984,0.006298,0.010663,0.986290,101694.718750,0.076191,1.131367e-08,1.479475e-08,2.029277e-08,0.007522,0.072511,1337.441284,100070.937500,0.210819,0.173185,0.133389,0.093220,0.053980,0.000888,0.000003,4.347277e-06,0.000009,4.880845e-09,3.887436e-06,0.009879,0.000005,0.000003,4.690559e-07,0.006586,0.000009,6.928641e-07,3.153174e-06,25.591148,1.778901e-07,7.377920e-10,1.319926e-09,4.948856e-10,5.604479e-10,4.120476e-09,8.757213e-10,1.897001e-10,7.463049e-09,1.960385e-10,3.993148e-09,2.022502e-14,2.141237e-08,8.628354e-10,7.536546e-09,6.557893e-10,5.894945e-08,3.017641e-09,2.562929e-10,2.378038e-11,1.905661e-09,2.728035e-10,0.009295,2.539864e-09,6.091789e-09,290.919830,3.897127,2.624408,0.878641,0.005784,25.642176,2.074938e-06,4.218031e-06,3.245817e-06,1.298927e-07,0.000003,5.384176e-07,0.000013,2.288023e-07,0.000016,9.983653e-07,0.000008,1.850165e-06
1,1003,2010,77.826608,0.065638,0.113299,286.096985,292.149780,0.006543,0.013571,0.793853,101686.296875,0.074374,1.082756e-08,1.474964e-08,2.021521e-08,0.010043,0.070549,426.965088,101166.343750,0.211824,0.175080,0.136272,0.097289,0.058477,0.000877,0.000003,3.707628e-06,0.000009,4.924390e-09,2.554849e-06,0.009991,0.000005,0.000003,4.425227e-07,0.006451,0.000008,6.634551e-07,2.463177e-06,27.551956,1.661377e-07,8.725340e-10,1.541212e-09,6.062684e-10,5.508705e-10,3.224980e-09,9.064102e-10,2.039385e-10,6.913703e-09,2.441078e-10,3.649391e-09,2.652614e-14,1.445367e-08,9.667648e-10,6.547783e-09,5.070752e-10,6.086813e-08,2.466012e-09,2.234096e-10,4.787582e-11,3.910108e-09,1.548187e-09,0.010791,2.249294e-09,3.825508e-09,292.066528,3.845286,2.597180,0.879368,0.005067,27.597118,2.609881e-06,5.381241e-06,4.267411e-06,1.304158e-07,0.000002,5.643739e-07,0.000013,2.814684e-07,0.000020,1.889070e-06,0.000008,1.206832e-06
2,1005,2010,75.841973,0.336253,-0.075642,284.153351,291.171997,0.006456,0.010674,0.987881,101689.679688,0.074089,1.173599e-08,1.520591e-08,2.082824e-08,0.007945,0.067335,1178.228027,100259.171875,0.202453,0.166499,0.128529,0.090249,0.052785,0.000884,0.000003,4.157243e-06,0.000009,4.883077e-09,3.436399e-06,0.009900,0.000005,0.000003,4.421493e-07,0.006519,0.000009,6.808563e-07,2.526453e-06,26.079842,1.840146e-07,7.171533e-10,1.263283e-09,4.802824e-10,5.927475e-10,4.009649e-09,8.788015e-10,2.286225e-10,7.826896e-09,3.231082e-10,4.440267e-09,2.134459e-14,2.074101e-08,8.272189e-10,6.289255e-09,3.875605e-10,5.690426e-08,2.879866e-09,2.500995e-10,2.469199e-11,1.982629e-09,3.230463e-10,0.009707,2.009621e-09,2.205164e-09,291.179504,4.070218,2.398671,0.878966,0.006158,26.123323,2.068363e-06,4.211733e-06,3.332815e-06,1.467424e-07,0.000002,5.382911e-07,0.000013,2.391392e-07,0.000017,1.143327e-06,0.000007,1.193010e-06
3,1007,2010,73.705432,0.221962,-0.004751,283.564850,290.792206,0.006248,0.010553,0.986778,101695.648438,0.077052,1.129947e-08,1.473398e-08,2.021792e-08,0.007260,0.072814,1267.733643,100154.929688,0.211800,0.173926,0.133868,0.093414,0.053924,0.000892,0.000003,4.335408e-06,0.000009,4.883221e-09,3.874136e-06,0.009887,0.000005,0.000003,4.890971e-07,0.006616,0.000009,6.987318e-07,3.148527e-06,25.509922,1.782282e-07,7.250749e-10,1.290446e-09,4.728918e-10,5.571377e-10,4.112687e-09,8.437939e-10,1.833033e-10,7.471534e-09,1.787300e-10,3.976964e-09,2.046940e-14,2.232927e-08,8.603168e-10,8.605236e-09,9.747133e-10,5.742021e-08,2.987445e-09,2.592347e-10,2.208323e-11,1.767885e-09,2.444768

In [12]:
final_df = final_df.rename(columns=
        {'u10': '10m u-component of wind', 'v10': '10m v-componend of wind', 'd2m':'2m dew point temperature',  
          't2m':'2m temperature', 'bcaod550':'Black carbon AOD at 550 nm','duaod550':'Dust AOD at 550 nm',                                                  
          'lsm':'Land-sea mask', 'msl':'Mean sea level pressure', 'omaod550':'Organic matter AOD at 550 nm',
          'pm1':'PM$_1$', 'pm2p5':'PM$_{2.5}$', 'pm10':'PM$_{10}$', 'ssaod550':'Sea salt AOD at 550 nm',
          'suaod550':'Sulphate AOD at 550 nm', 'z':'Surface geopotential', 'sp':'Surface pressure',  
          'aod469':'Total AOD at 469 nm', 'aod550':'Total AOD at 550 nm', 'aod670':'Total AOD at 670 nm',
          'aod865':'Total AOD at 865 nm', 'aod1240': 'Total AOD at 1240 nm', 'tcco':'Total column carbon monoxide',
          'tc_c2h6':'Total column ethane', 'tchcho':'Total column formaldehyde', 'tc_h2o2':'Total column hydrogen peroxide',
          'tc_oh':'Total column hydroxyl radical', 'tc_c5h8':'Total column isoprene', 'tc_ch4':'Total column methane',
          'tc_hno3':'Total column nitric acid', 'tcno2':'Total column nitrogen dioxide', 'tc_no':'Total column nitrogen monoxide',
          'gtco3':'Total column ozone', 'tc_pan':'Total column peroxyacetyl nitrate','tc_c3h8':'Total column propane',
          'tcso2':'Total column sulphur dioxide', 'tcwv':'Total column water vapour', 'co':'Carbon monoxide',
          'aermr04':'Dust aerosol (0.03-0.55 \u00B5m) mixing ratio', 'aermr05':'Dust aerosol (0.55-0.9 \u00B5m) mixing ratio',
          'aermr06':'Dust aerosol (0.9-20 \u00B5m) mixing ratio', 'c2h6':'Ethane', 'hcho':'Formaldehyde',
          'h2o2':'Hydrogen peroxide', 'aermr09':'Hydrophilic black carbon aerosol mixing ratio', 
          'aermr07':'Hydrophilic organic matter aerosol mixing ratio', 
          'aermr10':'Hydrophobic black carbon aerosol mixing ratio',
          'aermr08':'Hydrophobic organic matter aerosol mixing ratio', 'oh':'Hydroxyl radical',
          'c5h8':'Isoprene', 'hno3':'Nitric acid', 'no2':'Nitrogen dioxide', 'no':'Nitrogen monoxide', 'go3':'Ozone',
          'pan':'Peroxyacetyl nitrate', 'c3h8':'Propane', 'aermr01':'Sea salt aerosol (0.03-0.5 \u00B5m) mixing ratio',
          'aermr02':'Sea salt aerosol (0.5-5 \u00B5m) mixing ratio', 'aermr03':'Sea salt aerosol (5-20 \u00B5m) mixing ratio',
          'q':'Specific humidity', 'aermr11':'Sulphate aerosol mixing ratio', 'so2':'Sulphur dioxide',
          't':'Temperature', 'lai_hv':'Leaf area index, high vegetation', 'lai_lv':'Leaf area index, low vegetation',
          'asn':'Snow albedo', 'sd':'Snow depth', 'tcw':'Total column water',
          'aermssdus':'Vertically integrated mass of dust aerosol (0.03-0.55 \u00B5m)',
         'aermssdum':'Vertically integrated mass of dust aerosol (0.55-9 \u00B5m)',
         'aermssdul':'Vertically integrated mass of dust aerosol (9-20 \u00B5m)',
         'aermssbchphil':'Vertically integrated mass of hydrophilic black carbon aerosol',
         'aermssomhphil':'Vertically integrated mass of hydrophilic organic matter aerosol',
         'aermssbchphob':'Vertically integrated mass of hydrophobic black carbon aerosol',
         'aermssomhphob':'Vertically integrated mass of hydrophobic organic matter aerosol',
         'aermsssss':'Vertically integrated mass of sea salt aerosol (0.03-0.5 \u00B5m)',
         'aermssssm':'Vertically integrated mass of sea salt aerosol (0.5-5 \u00B5m)',
         'aermssssl':'Vertically integrated mass of sea salt aerosol (5-20 \u00B5m)',
         'aermsssu':'Vertically integrated mass of sulphate aerosol',
         'aermssso2':'Vertically integrated mass of sulphur dioxide',         
        })
final_df

,fips,year,MeanLifeExpectency,10m u-component of wind,10m v-componend of wind,2m dew point temperature,2m temperature,Black carbon AOD at 550 nm,Dust AOD at 550 nm,Land-sea mask,Mean sea level pressure,Organic matter AOD at 550 nm,PM$_1$,PM$_{2.5}$,PM$_{10}$,Sea salt AOD at 550 nm,Sulphate AOD at 550 nm,Surface geopotential,Surface pressure,Total AOD at 469 nm,Total AOD at 550 nm,Total AOD at 670 nm,Total AOD at 865 nm,Total AOD at 1240 nm,Total column carbon monoxide,Total column ethane,Total column formaldehyde,Total column hydrogen peroxide,Total column hydroxyl radical,Total column isoprene,Total column methane,Total column nitric acid,Total column nitrogen dioxide,Total column nitrogen monoxide,Total column ozone,Total column peroxyacetyl nitrate,Total column propane,Total column sulphur dioxide,Total column water vapour,Carbon monoxide,Dust aerosol (0.03-0.55 µm) mixing ratio,Dust aerosol (0.55-0.9 µm) mixing ratio,Dust aerosol (0.9-20 µm) mixing ratio,Ethane,Formaldehyde,Hydrogen peroxide,Hydrophilic black carbon aerosol mixing ratio,Hydrophilic organic matter aerosol mixing ratio,Hydrophobic black carbon aerosol mixing ratio,Hydrophobic organic matter aerosol mixing ratio,Hydroxyl radical,Isoprene,Nitric acid,Nitrogen dioxide,Nitrogen monoxide,Ozone,Peroxyacetyl nitrate,Propane,Sea salt aerosol (0.03-0.5 µm) mixing ratio,Sea salt aerosol (0.5-5 µm) mixing ratio,Sea salt aerosol (5-20 µm) mixing ratio,Specific humidity,Sulphate aerosol mixing ratio,Sulphur dioxide,Temperature,"Leaf area index, high vegetation","Leaf area index, low vegetation",Snow albedo,Snow depth,Total column water,Vertically integrated mass of dust aerosol (0.03-0.55 µm),Vertically integrated mass of dust aerosol (0.55-9 µm),Vertically integrated mass of dust aerosol (9-20 µm),Vertically integrated mass of hydrophilic black carbon aerosol,Vertically integrated mass of hydrophilic organic matter aerosol,Vertically integrated mass of hydrophobic black carbon aerosol,Vertically integrated mass of hydrophobic organic matter aerosol,Vertically integrated mass of sea salt aerosol (0.03-0.5 µm),Vertically integrated mass of sea salt aerosol (0.5-5 µm),Vertically integrated mass of sea salt aerosol (5-20 µm),Vertically integrated mass of sulphate aerosol,Vertically integrated mass of sulphur dioxide
0,1001,2010,75.728489,0.244891,-0.022881,283.480743,291.070984,0.006298,0.010663,0.986290,101694.718750,0.076191,1.131367e-08,1.479475e-08,2.029277e-08,0.007522,0.072511,1337.441284,100070.937500,0.210819,0.173185,0.133389,0.093220,0.053980,0.000888,0.000003,4.347277e-06,0.000009,4.880845e-09,3.887436e-06,0.009879,0.000005,0.000003,4.690559e-07,0.006586,0.000009,6.928641e-07,3.153174e-06,25.591148,1.778901e-07,7.377920e-10,1.319926e-09,4.948856e-10,5.604479e-10,4.120476e-09,8.757213e-10,1.897001e-10,7.463049e-09,1.960385e-10,3.993148e-09,2.022502e-14,2.141237e-08,8.628354e-10,7.536546e-09,6.557893e-10,5.894945e-08,3.017641e-09,2.562929e-10,2.378038e-11,1.905661e-09,2.728035e-10,0.009295,2.539864e-09,6.091789e-09,290.919830,3.897127,2.624408,0.878641,0.005784,25.642176,2.074938e-06,4.218031e-06,3.245817e-06,1.298927e-07,0.000003,5.384176e-07,0.000013,2.288023e-07,0.000016,9.983653e-07,0.000008,1.850165e-06
1,1003,2010,77.826608,0.065638,0.113299,286.096985,292.149780,0.006543,0.013571,0.793853,101686.296875,0.074374,1.082756e-08,1.474964e-08,2.021521e-08,0.010043,0.070549,426.965088,101166.343750,0.211824,0.175080,0.136272,0.097289,0.058477,0.000877,0.000003,3.707628e-06,0.000009,4.924390e-09,2.554849e-06,0.009991,0.000005,0.000003,4.425227e-07,0.006451,0.000008,6.634551e-07,2.463177e-06,27.551956,1.661377e-07,8.725340e-10,1.541212e-09,6.062684e-10,5.508705e-10,3.224980e-09,9.064102e-10,2.039385e-10,6.913703e-09,2.441078e-10,3.649391e-09,2.652614e-14,1.445367e-08,9.667648e-10,6.547783e-09,5.070752e-10,6.086813e-08,2.466012e-09,2.234096e-10,4.787582e-11,3.910108e-09,1.548187e-09,0.010791,2.249294e-09,3.825508e-09,292.066528,3.845286,2.597180,0.879368,0.005067,27.5